# ALGORITHMIC QUESTION
----
### A number n of kids are in a camp. Between some k pairs of them (a kid can be part of more than one pairs) there are often fights. At night there are two dormitories where the kids can sleep. We want, if possible, to assign each kid in one of the two dormitories in such a way that each pair of kids that fights often is assigned to a different dormitory. (There are no space problems and the two dormitories can have different number of kids.)

### Give an algorithm that is linear in n and k that is able to answer whether such an assignment is possible and, if so, return one.

So the problem has this structure:
- n = number of kids
- k = fight pair 
- m = 2 dormitories 
- $ min:f(x) =$ fighting level (number of fighting pair in each dormitories)

it is clear that we have : based on the binomial coefficient $\binom{n}{k} = \frac{n!}{k!(n-k)!}$ 

with $n \in \mathbb{N}$ and $k = 2$. (i.g. for 10 kids we have at most $\binom{10}{2}= 45$ possible pair of enemies)

Our algorithm will be possibly accept as an input a list/dictionary of the possible pairs of all the fights between the kids, and will return a optimal disposition into the two dormitories possibly, after minimazing the $f(x)$ it will also balance the presence of kids in each dormitories.


we will be using a dictionary where the keys are the id of each kid and its values will be the id of the kids whom each kid has a threat.

Let's create a function which will generate random possible fight lists:

In [1]:
import random                       #importing random library for random initialization
from collections import defaultdict   #importing default dict for using it as {key : [values]}
                                       #since lists as values are useful in this task

In [2]:
random.seed(15)         #setting seed for reproducibility
n = 10 
k = '?'

In [3]:
def fightdict(n):
    
    f_dict = defaultdict(list)                                 #initializing empty dict 
    
    for key in range(1,n+1):                                              #creating fight dictionary
        f_dict[key] = random.sample(range(1, n+1), random.randint(0, n))   #randomly picking enmity for each kid
        for value in f_dict[key]:                                        #removing enmity against the kid himself
            if value == key:
                f_dict[key].remove(value)
                
    return f_dict    

def adjustdict(d):
    for key in range(1,len(d)):                          #we're removing absent enmity match (make love >.<)
        for value in d[key]:            #based on the assumption if you have problems with me the problem is yours
            if key not in d[value]:
                d[key].remove(value)
    return d

In [4]:
def adjustdict(d):                    #adjusting the enmity dict by removing enmities that are not mutual
    for key in d.keys():
        l = []
        for value in d[key]:
            if key in d[value]:
                l.append(value)
        d[key] = l 
        l = []
    
    return d

So this is our fighiting dict before adjustments:

In [5]:
a = fightdict(10)
a

defaultdict(list,
            {1: [9, 10],
             2: [4, 1],
             3: [],
             4: [3, 6, 1, 10, 8, 9, 2, 5, 7],
             5: [4, 10, 6, 7, 3],
             6: [5, 9, 7],
             7: [10, 8],
             8: [2, 6, 4, 5, 3, 7],
             9: [2, 8, 1, 10, 7, 5],
             10: [1]})

This is the adjusted dict, as is possible to see non mutual enmity have been deleted 

In [6]:
adjustdict(a)  

defaultdict(list,
            {1: [9, 10],
             2: [4],
             3: [],
             4: [8, 2, 5],
             5: [4, 6],
             6: [5],
             7: [8],
             8: [4, 7],
             9: [1],
             10: [1]})

Now that we have our dictionary to label kids fightings let's move on :
- think about the fact: if one doesn't get along only with 9,10 this means is going to be fine with anyone else

In [7]:
def Nofightings(d):
    
    dorm1 , dorm2 = {} , {}                        #initializing empty dormitory dict
    fighting_lv1 , fighting_lv2 = 0 , 0            #0 level of fighitings in dormitory 
   
    
    '''CALCULATING FIGHTING LEVEL FOR EACH DORMITORY'''
    '''AND RANDOMLY FILLING THE DORMITORY'''
    
    for kid in d:
        dorm_picker = random.randint(1,2)            #randomly allocating kids in one dormitory
        if kid not in dorm1 and dorm_picker == 1:
            dorm1[kid] = d[kid]
            
        else:
            dorm2[kid] = d[kid]
    
    for kid in dorm1:                          #checking the enmity for each kid for both dormitory:
        for enemy in dorm1[kid]:               #if a corresponding enemy is in the same dormitory of the kid 
            if enemy in dorm1:                 #the fighitning level of the dormitory increases 
                fighting_lv1 +=1
                
    for kid in dorm2:
        for enemy in dorm2[kid]:
            if enemy in dorm2:
                fighting_lv2 +=1       
            
    fighting_lv2 = fighting_lv2/2              #dividing by 2 to count just one time the enmity 
    fighting_lv1 = fighting_lv1/2              #(otherwise) it would have been counted twice one for each kid
    
    
    
    print('Random allocation for Dormitory 1', dorm1,fighting_lv1)
    print('Random allocation for Dormitory 2', dorm2,fighting_lv2)    #checking the initial situation for comparing
    
    
    '''re-allocating kids checking enmity'''
    
    for kid in dorm1.copy():                         #checking for each kid in the dormitory if the enemy is in the
        if dorm1[kid] == []:                         #same dormitory, if so checking the most troublemaker 
            pass                                     #by comparing the len of the enmity list and just swap the
        for enemy in dorm1[kid]:                                           #kinder kid in the other dormitory
            if enemy in dorm1 and len(dorm1[kid]) <= len(dorm1[enemy]):            
                dorm2[kid] = dorm1[kid]
                dorm1.pop(kid)
                fighting_lv1 -=1                           #since the threath for that specific kid has been solved
                                                         #the fighting level decreases
        
    for kid in dorm2.copy():
        if dorm2[kid] == []:
            pass
        for enemy in dorm2[kid]:
            if enemy in dorm2 and len(dorm2[kid]) <= len(dorm2[enemy]):            
                dorm1[kid] = dorm2[kid]
                dorm2.pop(kid)
                fighting_lv2 -=1
                
    
    if fighting_lv1 < 0:                                 #if the fighting level gets less than 0 round it to 0 
        fighting_lv1 = 0
    if fighting_lv2 < 0:
        fighting_lv2 = 0
    
    #return dorm1,int(fighting_lv1),dorm2,int(fighting_lv2)
    print('Dormitory 1 will host kid : ', dorm1.keys(),end = '\t')
    print('The expected level of fighitings is : ', int(fighting_lv1))
    print('Dormitory 2 will host kid : ', dorm2.keys(),end = '\t')
    print('The expected level of fighitings is : ', int(fighting_lv2))

In [8]:
Nofightings(a)

Random allocation for Dormitory 1 {3: [], 4: [8, 2, 5], 5: [4, 6], 7: [8], 8: [4, 7]} 3.0
Random allocation for Dormitory 2 {1: [9, 10], 2: [4], 6: [5], 9: [1], 10: [1]} 2.0
Dormitory 1 will host kid :  dict_keys([3, 4, 6, 9, 10, 7])	The expected level of fighitings is :  0
Dormitory 2 will host kid :  dict_keys([1, 2, 5, 8])	The expected level of fighitings is :  0
